# Prepare Ellsberg data

This notebook generates 'data_ellsberg.csv'. It processes the raw data and provides a standardised output:
- Cleans up the results of the emotion elicitation questionnaire
- Calculates a single locus score for each participant - See [original paper](https://psycnet.apa.org/record/2011-19211-001), page 11.
- Calculates a Big 5 score (openness to experience, conscientiousness, extraversion, agreeableness, and emotional stability) for each participant - See [original paper](https://www.sciencedirect.com/science/article/abs/pii/S0092656603000461), appendix A.

Notes:
- The "primary ID" for our dataset throughout the entire process stays the index of the entry in `raw_data_ellsberg.csv`

Exact cleaning steps:
1. Remove duplicate column headers
2. Remove exactly 3 participants with low progress (not enough data)
3. Rename generic column names into useable ones
4. Remove participant with multiple records (there should be exactly 1 participant to remove)
5. Join against demographic data
6. Convert game choice data types
7. Convert certainty values data types 
8. Convert game emotions values data types
9. Convert TIPI answer data types
10. Calculate TIPI values
11. Calculate locus values
12. Add a column saying if the participant displays ambiguity aversion (DAA)
13. Write processed output CSV

In [ ]:
import numpy as np
import pandas as pd

ellsberg = 'data/raw_data_ellsberg.csv'
demographic = 'data/raw_data_demographic.csv'
output = 'data/data_ellsberg.csv'

df = pd.read_csv(ellsberg)
demographic_df = pd.read_csv(demographic)

#### Step 1: Remove duplicate column headers

In [ ]:
df = df.drop([0])

#### Step 2: Remove exactly 3 participants with low progress (not enough data)

In [ ]:
df = df.astype({'Progress': 'int32'})
before = len(df)
df = df[df['Progress'] >= 99]
after = len(df)
print(f'Removed {before - after} participants with low progess. Records left: {after}')

#### Step 3: Rename generic column names into useable ones

In [ ]:
mapping = {
    "Q78": "participant_id", 

    "Q7": "ChoiceGame1",
    "Q72": "CertaintyGame1",
    "Q57_1": "Hopeful1A",
    "Q58_1": "Curiosity1A",
    "Q59_1": "Enlightenment1A",
    "Q54_1": "Thrilled1A",
    "Q61_1": "Anticipatory1A",
    "Q56_1": "Satisfied1A",
    "Q63_1": "Hopeful1B",
    "Q64_1": "Curiosity1B",
    "Q65_1": "Enlightenment1B",
    "Q66_1": "Thrilled1B",
    "Q67_1": "Anticipatory1B",
    "Q68_1": "Satisfied1B",
    "Q13": "ChoiceGame2",
    "Q14": "CertaintyGame2",
    "Q15_1": "Hopeful2A",
    "Q52_1": "Curiosity2A",
    "Q53_1": "Enlightenment2A",
    "Q60_1": "Thrilled2A",
    "Q55_1": "Anticipatory2A",
    "Q62_1": "Satisfied2A",
    "Q69_1": "Hopeful2B",
    "Q70_1": "Curiosity2B",
    "Q71_1": "Enlightenment2B",
    "Q72_1": "Thrilled2B",
    "Q73_1": "Anticipatory2B",
    "Q74_1": "Satisfied2B",
    "Q17_1": "TipiEnthusiastic",
    "Q17_2": "TipiQuiet",
    "Q17_3": "TipiWarm",
    "Q17_4": "TipiQuarrelsome",
    "Q17_5": "TipiDisciplined",
    "Q17_6": "TipiCareless",
    "Q17_7": "TipiStable",
    "Q17_8": "TipiAnxious",
    "Q17_9": "TipiComplex",
    "Q17_10": "TipiUncreative",

    "Q15": "Locus1",
    "Q17": "Locus2",
    "Q19": "Locus3",
    "Q21": "Locus4",
    "Q23": "Locus5",
    "Q25": "Locus6",
    "Q27": "Locus7",
    "Q29": "Locus8",
    "Q31": "Locus9",
    "Q33": "Locus10",
    "Q35": "Locus11",
    "Q37": "Locus12",
    "Q39": "Locus13",
    "Q41": "Locus14",
    "Q43": "Locus15",
    "Q45": "Locus16",
    "Q47": "Locus17",
    "Q49": "Locus18",
    "Q51": "Locus19",
    "Q53": "Locus20",
    "Q55": "Locus21",
    "Q57": "Locus22",
    "Q59": "Locus23",
    "Q61": "Locus24",
    "Q63": "Locus25",
    "Q65": "Locus26",
    "Q67": "Locus27",
    "Q69": "Locus28",
    "Q71": "Locus29",
}

df = df[mapping.keys()]
df = df.rename(columns=mapping)

#### Step 4: Remove participant with multiple records (there should be exactly 1 participant to remove)

In [ ]:
# Find participants with multiple records
count = df['participant_id'].value_counts()

before = len(df)

# Remove all records from participants with more than one record
for id, n in count[count > 1].items():
    print(f'Removing {n} records for {id}')
    df = df[df['participant_id'] != id]

after = len(df)

print(f'Removed {before - after} records from same participants. Records left: {after}')

#### Step 5: Join against demographic data

In [ ]:
demographic_cols = ['participant_id', 'Sex', 'Age']
demographic_df = demographic_df[demographic_cols]

# Add the index of 'df' as a new column
before = len(df)

df['index'] = df.index
df = pd.merge(df, demographic_df, on='participant_id')

# Restore index from original 'df'
df.index = df['index']
df = df.drop(columns=['index'])

after = len(df)
print(f'Removed {before - after} participants without demographic information. Records left: {after}')

#### Step 6: Convert game choice data types

If the participant chose lottery A, set the value as (numeric) 1, otherwise, set the value as (numeric) 2

In [ ]:
choice_cols = ['ChoiceGame1', 'ChoiceGame2']
for col in choice_cols:
    df[col] = np.where(df[col] == 'Lottery A', 1, 2)

choice_dtype = {c: 'int32' for c in choice_cols}
df = df.astype(choice_dtype)

#### Step 7: Convert game certainty data types

Remove text values from game certainty columns, and cast to numeric types

In [ ]:
# Canonicalize values for game certainty

certainty_cols = ['CertaintyGame1', 'CertaintyGame2']
for col in certainty_cols:
    df = df.replace({col: {'Very Uncertain\n1': 1, 'Very Certain\n5': 5}})

certainty_dtype = {c: 'int32' for c in certainty_cols}
df = df.astype(certainty_dtype)

#### Step 8: Convert game emotions data types

For every emotion (hopeful, curiosity, enlightment, thrilled, anticipatory, satisfied), replace categorical values (enums) with numeric ones (1-4)

In [ ]:
hopeful_columns = [ 'Hopeful1A', 'Hopeful1B', 'Hopeful2A', 'Hopeful2B' ]
curiosity_columns = [ 'Curiosity1A', 'Curiosity1B', 'Curiosity2A', 'Curiosity2B' ]
enlightenment_columns = [ 'Enlightenment1A', 'Enlightenment1B', 'Enlightenment2A', 'Enlightenment2B' ]
thrilled_columns = [ 'Thrilled1A', 'Thrilled1B', 'Thrilled2A', 'Thrilled2B' ]
anticipatory_columns = [ 'Anticipatory1A', 'Anticipatory1B', 'Anticipatory2A', 'Anticipatory2B' ]
satisfied_columns = [ 'Satisfied1A', 'Satisfied1B', 'Satisfied2A', 'Satisfied2B' ]

hopeful_values = {'Worry': 1, 'Discomfort': 2, 'Comfort': 3, 'Hopeful': 4}
curiosity_values = {'Boredom': 1, 'Indifference': 2, 'Interest': 3, 'Curiosity': 4}
enlightenment_values = {'Puzzlement': 1, 'Confusion': 2, 'Insight': 3, 'Enlightenment': 4}
thrilled_values = {'Disappointed': 1, 'Dissatisfied': 2, 'Satisfied': 3, 'Thrilled': 4}
anticipatory_values = {'Dread': 1, 'Apprehension': 2, 'Calm': 3, 'Anticipatory': 4}
satisfied_values = {'Embarrassed': 1, 'Self-conscious': 2, 'Pleased': 3, 'Satisfied': 4}

mapping = [
    (hopeful_columns, hopeful_values),
    (curiosity_columns, curiosity_values),
    (enlightenment_columns, enlightenment_values),
    (thrilled_columns, thrilled_values),
    (anticipatory_columns, anticipatory_values),
    (satisfied_columns, satisfied_values)
]

for columns, values in mapping:
    numeric_mapping = {column: values for column in columns}
    df = df.replace(numeric_mapping)

#### Step 9: Convert TIPI answer data types

Remove textual values and cast to numeric types

In [ ]:
tipi_cols = [
    "TipiEnthusiastic",
    "TipiQuiet",
    "TipiWarm",
    "TipiQuarrelsome",
    "TipiDisciplined",
    "TipiCareless",
    "TipiStable",
    "TipiAnxious",
    "TipiComplex",
    "TipiUncreative",
]

# Remove strings for '1' and '7' values
for col in tipi_cols:
    df = df.replace({col: {'1 strongly\ndisagree': 1, '7 strongly\nagree': 7}})

# Update tipi columns to be integers
tipi_dtype_mapping = {col: 'int32' for col in tipi_cols}
df = df.astype(tipi_dtype_mapping)


#### Step 10: Calculate TIPI values

In [ ]:
# Map columns to the question numbers in the TIPI calculation document
tipi_mapping = {
    "Tipi1": "TipiEnthusiastic",
    "Tipi6": "TipiQuiet",
    "Tipi7": "TipiWarm",
    "Tipi2": "TipiQuarrelsome",
    "Tipi3": "TipiDisciplined",
    "Tipi8": "TipiCareless",
    "Tipi9": "TipiStable",
    "Tipi4": "TipiAnxious",
    "Tipi5": "TipiComplex",
    "Tipi10": "TipiUncreative",
}

# Calculate Big5
big5 = ['Openess', 'Conciensiousness', 'Extroversion', 'Agreability', 'Stability']

def tipi_score(major, minor):
    major_col = tipi_mapping[major]
    minor_col = tipi_mapping[minor]
    return (df[major_col] + (8 - df[minor_col])) / 2

df['Openess'] = tipi_score('Tipi9', 'Tipi10')
df['Conciensiousness'] = tipi_score('Tipi5', 'Tipi6')
df['Extroversion'] = tipi_score('Tipi1', 'Tipi2')
df['Agreability'] = tipi_score('Tipi3', 'Tipi4')
df['Stability'] = tipi_score('Tipi7', 'Tipi8')

print(df[big5].mean())

# Averages from Journal:
# Conciensiousness = 4.61
# Agreability = 4.69
# Stability = 4.34
# Openess = 5.51
# Extroversion = 3.98

# Remove raw columns
df = df.drop(tipi_mapping.values(), axis=1)

#### Step 11: Calculate locus values

- Replaces categorical values (textual answers) with numerical values of 1 (first answer) and 2 (second answer)
- Adds a column per question that says if the participant should get a point for that answer or not
- Calculates a single column with the aggregated locus score per participant

Note: Questions 1, 8, 14, 19, 24 and 27 are not scored, see the Locus questionnaire for more information.

In [ ]:
# This mapping helps convert textual values to option they chose, i.e. a numeric 1 or 2
# Questions that are not scored simply don't appear here
locus_choice_mapping = {
    '1': 'Children',
    '2': 'Many',
    '3': 'One',
    '4': 'In',
    '5': 'The',
    '6': 'Without',
    '7': 'No',
    '8': 'Heredity',
    '9': 'I',
    '10': 'In',
    '11': 'Becoming',
    '12': 'The',
    '13': 'When',
    '14': 'There',
    '15': 'In',
    '16': 'Who',
    '17': 'As',
    '18': 'Most',
    '19': 'One',
    '20': 'It',
    '21': 'In',
    '22': 'With',
    '23': 'Sometimes',
    '24': 'A good leader expects',
    '25': 'Many',
    '26': 'People',
    '27': 'There',
    '28': 'What',
    '29': 'Most',
}

# This mapping helps convert participant answers to scoring, i.e. a numeric 1 if they are should get a point, otherwise 0
# Questions that are not scored are compared against '$', which will never appear, so the score will always be 0
locus_score_mapping = {
    '1': '$',
    '2': 'Many',
    '3': 'There',
    '4': 'Unfortunately',
    '5': 'Most',
    '6': 'Without',
    '7': 'No',
    '8': '$',
    '9': 'I',
    '10': 'Many',
    '11': 'Getting',
    '12': 'This',
    '13': 'It',
    '14': '$',
    '15': 'Many',
    '16': 'Who',
    '17': 'As',
    '18': 'Most',
    '19': '$',
    '20': 'It',
    '21': 'In',
    '22': 'It',
    '23': 'Sometimes',
    '24': '$',
    '25': 'Many',
    '26': 'There',
    '27': '$',
    '28': 'Sometimes',
    '29': 'Most',
}

# This conversion has to come first, as the next one overrides the original textual answers
for k, v in locus_score_mapping.items():
    locus_k = 'Locus' + k
    score_k = 'LocusScore' + k
    df[score_k] = np.where(df[locus_k].str.startswith(v, na=False), 1, 0)

# Override the textual answers with numerical values
for k, v in locus_choice_mapping.items():
    locus_k = 'Locus' + k
    df[locus_k] = np.where(df[locus_k].str.startswith(v, na=False), 1, 2)

# Calculate a single locus score column
score_keys = ['LocusScore' + dk for dk in locus_score_mapping]
df['Locus'] = df[score_keys].sum(axis=1)
df['Locus'].value_counts()

#### Step 12: Add a column saying if the participant displays ambiguity aversion (DAA)

In [ ]:
def isDAA(x):
    if x['ChoiceGame1'] == 1 and x['ChoiceGame2'] == 2:
        return 1
    return 0

df['DAA'] = df.apply(isDAA, axis=1)

#### Step 13: Write processed output CSV

In [ ]:
df.to_csv(output, index=True, index_label='ID')